<a href="https://colab.research.google.com/github/tarunmehrda/Facial-Keypoints-Detection/blob/main/TASK4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D,
    Dense, Dropout, Flatten,
    BatchNormalization
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [ ]:
train_df = pd.read_csv("training.csv")
test_df  = pd.read_csv("test.csv")
lookup   = pd.read_csv("IdLookupTable.csv")
sample_df=pd.read_csv('SampleSubmission.csv')

In [ ]:
train_df.head()

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,66.033564,39.002274,30.227008,36.421678,59.582075,39.647423,73.130346,39.969997,36.356571,37.389402,...,57.066803,61.195308,79.970165,28.614496,77.388992,43.312602,72.935459,43.130707,84.485774,238 236 237 238 240 240 239 241 241 243 240 23...
1,64.332936,34.970077,29.949277,33.448715,58.856170,35.274349,70.722723,36.187166,36.034723,34.361532,...,55.660936,56.421447,76.352000,35.122383,76.047660,46.684596,70.266553,45.467915,85.480170,219 215 204 196 204 211 212 200 180 168 178 19...
2,65.057053,34.909642,30.903789,34.909642,59.412000,36.320968,70.984421,36.320968,37.678105,36.320968,...,53.538947,60.822947,73.014316,33.726316,72.732000,47.274947,70.191789,47.274947,78.659368,144 142 159 180 188 188 184 180 167 132 84 59 ...
3,65.225739,37.261774,32.023096,37.261774,60.003339,39.127179,72.314713,38.380967,37.618643,38.754115,...,54.166539,65.598887,72.703722,37.245496,74.195478,50.303165,70.091687,51.561183,78.268383,193 192 193 194 194 194 193 192 168 111 50 12 ...
4,66.725301,39.621261,32.244810,38.042032,58.565890,39.621261,72.515926,39.884466,36.982380,39.094852,...,64.889521,60.671411,77.523239,31.191755,76.997301,44.962748,73.707387,44.227141,86.871166,147 148 160 196 215 214 216 217 219 220 206 18...


In [ ]:
test_df.shape

(1783, 2)

In [ ]:
sample_df.head()

,RowId,Location
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [ ]:
def load_images(df):
    X = df["Image"].apply(lambda x: np.array(x.split(), dtype=np.float32))
    X = np.stack(X) / 255.0
    X = X.reshape(-1, 96, 96, 1)
    return X

X = load_images(train_df)
X_test = load_images(test_df)


In [ ]:
y=train_df.drop(['Image'], axis=1).values
y=(y-48)/48

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.1, random_state=42
)


In [ ]:
inp = Input(shape=(96,96,1))

x = Conv2D(32, 3, padding="same", activation="relu")(inp)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)

x = Conv2D(64, 3, padding="same", activation="relu")(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)

x = Conv2D(128, 3, padding="same", activation="relu")(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)

x = Conv2D(256, 3, padding="same", activation="relu")(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)

x = Flatten()(x)

x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)

x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)

out = Dense(30)(x)

model = Model(inp, out)


In [ ]:
def masked_mse(y_true, y_pred):
    mask = tf.cast(~tf.math.is_nan(y_true), tf.float32)
    y_true = tf.where(tf.math.is_nan(y_true), y_pred, y_true)
    return tf.reduce_sum(mask * tf.square(y_true - y_pred)) / tf.reduce_sum(mask)


In [ ]:
model.compile(
    optimizer="adam",
    loss=masked_mse
)


In [ ]:
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=64,
    callbacks=[EarlyStopping(patience=8, restore_best_weights=True)],
    verbose=1
)

Epoch 1/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 285s 3s/step - loss: 0.1414 - val_loss: 1.4572
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 303s 3s/step - loss: 0.0205 - val_loss: 11.8334
Epoch 3/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 328s 3s/step - loss: 0.0102 - val_loss: 9.1327
Epoch 4/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 318s 3s/step - loss: 0.0094 - val_loss: 1.6177
Epoch 5/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 351s 3s/step - loss: 0.0094 - val_loss: 0.5556


In [ ]:
y_pred = model.predict(X_test)
y_pred = y_pred * 48 + 48
y_pred = np.clip(y_pred, 0, 96)


56/56 ━━━━━━━━━━━━━━━━━━━━ 15s 269ms/step


In [ ]:
lookup = pd.read_csv("IdLookupTable.csv")

feature_cols = train_df.drop(columns=["Image"]).columns.tolist()
feature_to_idx = {f: i for i, f in enumerate(feature_cols)}

locations = []
for _, row in lookup.iterrows():
    locations.append(
        y_pred[row["ImageId"] - 1, feature_to_idx[row["FeatureName"]]]
    )

submission = pd.DataFrame({
    "RowId": lookup["RowId"],
    "Location": locations
})

submission.to_csv("submission2.csv", index=False)
